In [1]:
import numpy as np
np.set_printoptions(formatter={'float': '{: 0.2f}'.format})
import pandas as pd
import math
import sys
%matplotlib inline

In [2]:
initial_distances=pd.read_excel('data.xlsx')
#Comment the below line when you load the actual distances.
initial_distances=initial_distances**0.5
print(initial_distances)
names=list(initial_distances.columns)

    A1   A2        A3        A4        A5        A6        A7        A8
0  0.0  5.0  6.000000  3.605551  7.071068  7.211103  8.062258  2.236068
1  NaN  0.0  6.082763  4.242641  5.000000  4.123106  3.162278  4.472136
2  NaN  NaN  0.000000  5.000000  1.414214  1.414214  7.280110  6.403124
3  NaN  NaN       NaN  0.000000  3.605551  4.123106  7.211103  1.414214
4  NaN  NaN       NaN       NaN  0.000000  1.414214  6.708204  5.000000
5  NaN  NaN       NaN       NaN       NaN  0.000000  5.385165  5.385165
6  NaN  NaN       NaN       NaN       NaN       NaN  0.000000  7.615773
7  NaN  NaN       NaN       NaN       NaN       NaN       NaN  0.000000


In [3]:
dists = np.array(initial_distances)
np.fill_diagonal(dists,0)
dists.shape

for i in range(0, dists.shape[0]):
    for j in range(i+1, dists.shape[1]):
        dists[j][i]=dists[i][j]
print(dists)

[[ 0.00  5.00  6.00  3.61  7.07  7.21  8.06  2.24]
 [ 5.00  0.00  6.08  4.24  5.00  4.12  3.16  4.47]
 [ 6.00  6.08  0.00  5.00  1.41  1.41  7.28  6.40]
 [ 3.61  4.24  5.00  0.00  3.61  4.12  7.21  1.41]
 [ 7.07  5.00  1.41  3.61  0.00  1.41  6.71  5.00]
 [ 7.21  4.12  1.41  4.12  1.41  0.00  5.39  5.39]
 [ 8.06  3.16  7.28  7.21  6.71  5.39  0.00  7.62]
 [ 2.24  4.47  6.40  1.41  5.00  5.39  7.62  0.00]]


In [4]:
def clusters_helper(arr,names):
    clusters={}
    for i in range(len(arr)):
        if arr[i] in clusters:
            clusters[arr[i]].append(names[i])
        else:
            clusters[arr[i]]=[]
            clusters[arr[i]].append(names[i])
    return clusters   

In [5]:
def find_clusters(dists,linkage,no_of_clusters = -1):
    data=dists.copy()
    iterations = 0
    if no_of_clusters<0:
        iterations = data.shape[0]
    else:
        iterations = data.shape[0]-no_of_clusters
        
    clusters = {}
    dis = {}
    row_index = -1
    col_index = -1
    array = []
    
    for n in range(data.shape[0]):
        array.append(n)
        
    clusters[0] = array.copy()
    print('Initially all points are seperate clusters ')
    print(clusters_helper(clusters[0],names))
    print('-----------------------------------------------------------------------')

    if iterations==0:
        print('-----------------------**Thank you**--------------------------------')
        return clusters
    
    for k in range(1, data.shape[0]):
        print('Iterating to form '+str(data.shape[0]-k)+' clusters')
        print(data)
        min_val = sys.maxsize
        for i in range(0, data.shape[0]):
            for j in range(0, data.shape[1]):
                #Checking validity of points and finding minimum distance
                if(data[i][j]>0 and data[i][j]<=min_val):
                    min_val = data[i][j]
                    row_index = i
                    col_index = j
                    
        print('Minimum value from the distance/dissimilarity matrix ',min_val)
        print('\n')
        #once we find the minimum value, we need to update the distance matrix
        #updating the matrix by calculating the new distances from the cluster to all points
        
        if(linkage == "single" or linkage =="Single"):
            for i in range(0,data.shape[0]):
                if(i != col_index):
                    temp = min(data[col_index][i],data[row_index][i])
                    data[col_index][i] = temp
                    data[i][col_index] = temp
        elif(linkage=="Complete" or linkage == "complete"):
             for i in range(0,data.shape[0]):
                if(i != col_index and i!=row_index):
                    temp = max(data[col_index][i],data[row_index][i])
                    data[col_index][i] = temp
                    data[i][col_index] = temp
        elif(linkage=="Average" or linkage == "average"):
             for i in range(0,data.shape[0]):
                if(i != col_index and i!=row_index):
                    temp = (data[col_index][i]+data[row_index][i])/2
                    data[col_index][i] = temp
                    data[i][col_index] = temp
                    
        # Making agglomerated points invalid         
        for i in range (0,data.shape[0]):
            data[row_index][i] = 0
            data[i][row_index] = 0

        # Updating cluster agglomeration
        minimum = min(row_index,col_index)
        maximum = max(row_index,col_index)
        for n in range(len(array)):
            if(array[n]==maximum):
                array[n] = minimum
        clusters[k] = array.copy()
        
        print(str(data.shape[0]-k)+' clusters')

        print(clusters_helper(clusters[k],names))
        print('-----------------------------------------------------------------------')

        if iterations==k:
            print(data)
            print('-----------------------**Thank you**--------------------------------')
            break
    return clusters

In [6]:
clusters_single_all=find_clusters(dists,'single')
clusters_single_all

Initially all points are seperate clusters 
{0: ['A1'], 1: ['A2'], 2: ['A3'], 3: ['A4'], 4: ['A5'], 5: ['A6'], 6: ['A7'], 7: ['A8']}
-----------------------------------------------------------------------
Iterating to form 7 clusters
[[ 0.00  5.00  6.00  3.61  7.07  7.21  8.06  2.24]
 [ 5.00  0.00  6.08  4.24  5.00  4.12  3.16  4.47]
 [ 6.00  6.08  0.00  5.00  1.41  1.41  7.28  6.40]
 [ 3.61  4.24  5.00  0.00  3.61  4.12  7.21  1.41]
 [ 7.07  5.00  1.41  3.61  0.00  1.41  6.71  5.00]
 [ 7.21  4.12  1.41  4.12  1.41  0.00  5.39  5.39]
 [ 8.06  3.16  7.28  7.21  6.71  5.39  0.00  7.62]
 [ 2.24  4.47  6.40  1.41  5.00  5.39  7.62  0.00]]
Minimum value from the distance/similarity matrix  1.4142135623730951


7 clusters
{0: ['A1'], 1: ['A2'], 2: ['A3'], 3: ['A4', 'A8'], 4: ['A5'], 5: ['A6'], 6: ['A7']}
-----------------------------------------------------------------------
Iterating to form 6 clusters
[[ 0.00  5.00  6.00  2.24  7.07  7.21  8.06  0.00]
 [ 5.00  0.00  6.08  4.24  5.00  4.12 

{0: [0, 1, 2, 3, 4, 5, 6, 7],
 1: [0, 1, 2, 3, 4, 5, 6, 3],
 2: [0, 1, 2, 3, 4, 4, 6, 3],
 3: [0, 1, 2, 3, 2, 2, 6, 3],
 4: [0, 1, 2, 0, 2, 2, 6, 0],
 5: [0, 1, 2, 0, 2, 2, 1, 0],
 6: [0, 1, 0, 0, 0, 0, 1, 0],
 7: [0, 0, 0, 0, 0, 0, 0, 0]}

In [7]:
clusters_single_till_4=find_clusters(dists,'single',no_of_clusters=4)
clusters_single_till_4

Initially all points are seperate clusters 
{0: ['A1'], 1: ['A2'], 2: ['A3'], 3: ['A4'], 4: ['A5'], 5: ['A6'], 6: ['A7'], 7: ['A8']}
-----------------------------------------------------------------------
Iterating to form 7 clusters
[[ 0.00  5.00  6.00  3.61  7.07  7.21  8.06  2.24]
 [ 5.00  0.00  6.08  4.24  5.00  4.12  3.16  4.47]
 [ 6.00  6.08  0.00  5.00  1.41  1.41  7.28  6.40]
 [ 3.61  4.24  5.00  0.00  3.61  4.12  7.21  1.41]
 [ 7.07  5.00  1.41  3.61  0.00  1.41  6.71  5.00]
 [ 7.21  4.12  1.41  4.12  1.41  0.00  5.39  5.39]
 [ 8.06  3.16  7.28  7.21  6.71  5.39  0.00  7.62]
 [ 2.24  4.47  6.40  1.41  5.00  5.39  7.62  0.00]]
Minimum value from the distance/similarity matrix  1.4142135623730951


7 clusters
{0: ['A1'], 1: ['A2'], 2: ['A3'], 3: ['A4', 'A8'], 4: ['A5'], 5: ['A6'], 6: ['A7']}
-----------------------------------------------------------------------
Iterating to form 6 clusters
[[ 0.00  5.00  6.00  2.24  7.07  7.21  8.06  0.00]
 [ 5.00  0.00  6.08  4.24  5.00  4.12 

{0: [0, 1, 2, 3, 4, 5, 6, 7],
 1: [0, 1, 2, 3, 4, 5, 6, 3],
 2: [0, 1, 2, 3, 4, 4, 6, 3],
 3: [0, 1, 2, 3, 2, 2, 6, 3],
 4: [0, 1, 2, 0, 2, 2, 6, 0]}

In [8]:
clusters_complete_all=find_clusters(dists,'complete')
clusters_complete_all

Initially all points are seperate clusters 
{0: ['A1'], 1: ['A2'], 2: ['A3'], 3: ['A4'], 4: ['A5'], 5: ['A6'], 6: ['A7'], 7: ['A8']}
-----------------------------------------------------------------------
Iterating to form 7 clusters
[[ 0.00  5.00  6.00  3.61  7.07  7.21  8.06  2.24]
 [ 5.00  0.00  6.08  4.24  5.00  4.12  3.16  4.47]
 [ 6.00  6.08  0.00  5.00  1.41  1.41  7.28  6.40]
 [ 3.61  4.24  5.00  0.00  3.61  4.12  7.21  1.41]
 [ 7.07  5.00  1.41  3.61  0.00  1.41  6.71  5.00]
 [ 7.21  4.12  1.41  4.12  1.41  0.00  5.39  5.39]
 [ 8.06  3.16  7.28  7.21  6.71  5.39  0.00  7.62]
 [ 2.24  4.47  6.40  1.41  5.00  5.39  7.62  0.00]]
Minimum value from the distance/similarity matrix  1.4142135623730951


7 clusters
{0: ['A1'], 1: ['A2'], 2: ['A3'], 3: ['A4', 'A8'], 4: ['A5'], 5: ['A6'], 6: ['A7']}
-----------------------------------------------------------------------
Iterating to form 6 clusters
[[ 0.00  5.00  6.00  3.61  7.07  7.21  8.06  0.00]
 [ 5.00  0.00  6.08  4.47  5.00  4.12 

{0: [0, 1, 2, 3, 4, 5, 6, 7],
 1: [0, 1, 2, 3, 4, 5, 6, 3],
 2: [0, 1, 2, 3, 4, 4, 6, 3],
 3: [0, 1, 2, 3, 2, 2, 6, 3],
 4: [0, 1, 2, 3, 2, 2, 1, 3],
 5: [0, 1, 2, 0, 2, 2, 1, 0],
 6: [0, 1, 0, 0, 0, 0, 1, 0],
 7: [0, 0, 0, 0, 0, 0, 0, 0]}

In [9]:
clusters_average_all=find_clusters(dists,'average')
clusters_average_all

Initially all points are seperate clusters 
{0: ['A1'], 1: ['A2'], 2: ['A3'], 3: ['A4'], 4: ['A5'], 5: ['A6'], 6: ['A7'], 7: ['A8']}
-----------------------------------------------------------------------
Iterating to form 7 clusters
[[ 0.00  5.00  6.00  3.61  7.07  7.21  8.06  2.24]
 [ 5.00  0.00  6.08  4.24  5.00  4.12  3.16  4.47]
 [ 6.00  6.08  0.00  5.00  1.41  1.41  7.28  6.40]
 [ 3.61  4.24  5.00  0.00  3.61  4.12  7.21  1.41]
 [ 7.07  5.00  1.41  3.61  0.00  1.41  6.71  5.00]
 [ 7.21  4.12  1.41  4.12  1.41  0.00  5.39  5.39]
 [ 8.06  3.16  7.28  7.21  6.71  5.39  0.00  7.62]
 [ 2.24  4.47  6.40  1.41  5.00  5.39  7.62  0.00]]
Minimum value from the distance/similarity matrix  1.4142135623730951


7 clusters
{0: ['A1'], 1: ['A2'], 2: ['A3'], 3: ['A4', 'A8'], 4: ['A5'], 5: ['A6'], 6: ['A7']}
-----------------------------------------------------------------------
Iterating to form 6 clusters
[[ 0.00  5.00  6.00  2.92  7.07  7.21  8.06  0.00]
 [ 5.00  0.00  6.08  4.36  5.00  4.12 

{0: [0, 1, 2, 3, 4, 5, 6, 7],
 1: [0, 1, 2, 3, 4, 5, 6, 3],
 2: [0, 1, 2, 3, 4, 4, 6, 3],
 3: [0, 1, 2, 3, 2, 2, 6, 3],
 4: [0, 1, 2, 0, 2, 2, 6, 0],
 5: [0, 1, 2, 0, 2, 2, 1, 0],
 6: [0, 1, 0, 0, 0, 0, 1, 0],
 7: [0, 0, 0, 0, 0, 0, 0, 0]}